In [448]:
import random as rand

In [449]:
# Variables importantes
numero_poblacion = 2
generaciones = 50
mochila = 10
probabilidad_cruza = 0.85
cruza_uniforme = 0.5

# Decoy detonators, love potion, extendable ears, skiving snackbox, fever fudge, puking pastilles, nosebleed nougat
ganancia = [10, 8, 12, 6, 3, 2, 2]
peso = [4, 2, 5, 5, 2, 1.5, 1]

In [450]:
# Generar población inicial
def poblacion(individuos):
    
    poblacion = []
    j = 0
    
    for i in range (individuos):
        
        individuo = [0,0,0,0,0,0,0]
        
        while j <= 6:
            if j == 1:
                individuo[1] = rand.randint(3,10)
            elif j == 3:
                individuo[3] = rand.randint(2,10)
            else:
                individuo[j] = rand.randint(0,10)
            j += 1 
        poblacion.append(individuo)
        j = 0
    
    return poblacion

In [451]:
def aptitud(poblacion):
    suma_total = 0
    suma_indv = [0]*numero_poblacion
    j = 0

    for individuo in poblacion:
        for i in range(7):
            suma_total += individuo[i] * ganancia[i]
            suma_indv[j] += individuo[i] * ganancia[i]
        j += 1

    # Calcular aptitud
    aptitud = []
    aptitud_acumulada = []

    for i in range (len(poblacion)):
        aptitud.append(suma_indv[i]/suma_total)
        
        if i == 0:
            aptitud_acumulada.append(aptitud[i])
        else:
            aptitud_acumulada.append(aptitud_acumulada[i-1]+aptitud[i])

    return aptitud_acumulada

In [452]:
def cruza(padres):
    
    var_aleatoria = []
    hijo1 = []
    hijo2 = []
    
    for i in range(7):
        var_aleatoria.append(rand.uniform(0,1))
    
    for i in range(7):
        if cruza_uniforme >= var_aleatoria[i]:
            hijo1.append(padres[0][i])
        else:
            hijo1.append(padres[1][i])
            
        if cruza_uniforme >= var_aleatoria[i]:
            hijo2.append(padres[1][i])
        else:
            hijo2.append(padres[0][i])
            
    return hijo1, hijo2

In [453]:
def ruleta(aptitud, poblacion):
    
    print(aptitud)
    
    padres = []
    random1 = rand.uniform(0,1)
    print(random1)    
    
    for i, apt in enumerate(aptitud):
        if apt >= random1 and poblacion[i] not in padres:
            padres.append(poblacion[i])
            break
            
    while len(padres) != 2:
        random2 = rand.uniform(0,1)
        for i, apt in enumerate(aptitud):
            if apt >= random2 and poblacion[i] not in padres:
                padres.append(poblacion[i])
                break

    print("Padres: ", padres)
        
    random_cruza = rand.uniform(0,1)
    if probabilidad_cruza >= random_cruza:
        return padres, True
    else:
        print("lol q mal")
        return padres, False

In [454]:
pob = poblacion(numero_poblacion)
print(pob)
apt = aptitud(pob)
x = 0
nueva_poblacion = []
while x < numero_poblacion/2:

    padres, flag = ruleta(apt, pob)

    if flag:
        hijo1, hijo2 = cruza(padres)
        nueva_poblacion.append(hijo1)
        nueva_poblacion.append(hijo2)

    else:
        nueva_poblacion.append(padres[0])
        nueva_poblacion.append(padres[1])

    x += 1

print("nueva_poblacion: ", nueva_poblacion)

[[0, 7, 2, 6, 5, 4, 3], [0, 3, 8, 8, 7, 3, 8]]
[0.40730337078651685, 1.0]
0.44445015545259925
Padres:  [[0, 3, 8, 8, 7, 3, 8], [0, 7, 2, 6, 5, 4, 3]]
nueva_poblacion:  [[0, 3, 8, 8, 7, 4, 3], [0, 7, 2, 6, 5, 3, 8]]
